In [1]:
import os
import pathlib
import numpy as np
import zipfile
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
import pathlib
import sys
import subprocess
# import tensorflow_io as tfio

In [ ]:
def extract_zip(zip_path, extract_to):
    zip_path_str = str(zip_path)
    
    if not zip_path_str.endswith('.zip'):
        zip_path_str += '.zip'
    
    zip_file_path = pathlib.Path(zip_path_str)
    
    folder_name = zip_file_path.stem 
    target_folder = pathlib.Path(extract_to) / folder_name
    
    if target_folder.exists():
        print(f"Das Verzeichnis {target_folder} existiert bereits. Überspringe das Extrahieren.")
    else:
        if zip_file_path.exists():
            print(f"Extrahiere die Zip-Datei {zip_file_path} nach {extract_to}.")
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
            print(f"Zip-Datei {zip_file_path} erfolgreich extrahiert.")
        else:
            print(f"Die Zip-Datei {zip_file_path} existiert nicht.")

def rename_audio_files(root_path):
    for root, dirs, files in os.walk(root_path):
        parent_folder = os.path.basename(root)
        for file in files:
            if not file.startswith(f"{parent_folder}_"):
                if file.endswith(('.wav', '.mp3')):  
                    
                    old_file_path = os.path.join(root, file)
                    new_file_name = f"{parent_folder}_{file}"
                    new_file_path = os.path.join(root, new_file_name)
                        
                    os.rename(old_file_path, new_file_path)
        print(f"renaming of {root_path}/{parent_folder} complete")

In [3]:
def convert_wav(input_wav, output_wav, sample_rate=44100, channels=1):
    """
    Convert WAV file to a standardized format (Mono, 16kHz)
    """
    command = [
        'ffmpeg', '-i', input_wav, '-ar', str(sample_rate), '-ac', str(channels), output_wav
    ]
    subprocess.run(command, check=True)

def load_wav_file_basic(file_path):
    try:
        # Try to decode the WAV file
        with open(file_path, "rb") as f:
            wav_data = f.read()
        
        audio, sample_rate = tf.audio.decode_wav(wav_data, desired_channels=1, desired_samples=44100)
        print(f"File '{file_path}' successfully decoded!")
        return audio, sample_rate

    except Exception as e:
        print(f"Error decoding WAV file '{file_path}': {e}")
        print("Attempting to convert the file...")

        # Path for the converted file
        converted_file = "converted_temp.wav"
        
        # Convert the WAV file
        convert_wav(file_path, converted_file)
        
        # Load the converted file and try decoding again
        with open(converted_file, "rb") as f:
            wav_data = f.read()
        
        audio, sample_rate = tf.audio.decode_wav(wav_data, desired_channels=1, desired_samples=44100)
        print(f"Converted file '{file_path}' successfully decoded!")
        e
        os.remove(converted_file)
        
        return audio, sample_rate

def process_directory_basic(directory_path):
    wav_files = []

    # Walk through all directories and subdirectories
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.lower().endswith(".wav"):
                wav_files.append(os.path.join(root, file))

    if not wav_files:
        print("No WAV files found in the directory.")
        return

    for file_path in wav_files:
        print(f"Processing file: {file_path}")
        audio, sample_rate = load_wav_file_basic(file_path)
        print(f"Audio shape: {audio.shape}, Sample rate: {sample_rate}")

In [4]:
def load_wav_file_io(file_path):
    try:
        # Lade die Audiodaten mit tensorflow-io (unterstützt 24-bit WAV)
        audio = tfio.audio.AudioIOTensor(file_path)
        print(audio)
        
        
        data = tf.cast(audio.to_tensor(), tf.float32) 
        sample_rate = audio.rate.numpy()
        
        print(f"File '{file_path}' loaded successfully! Sample rate: {sample_rate}, Channels: {audio.shape[-1]}")
        
        if audio.shape[-1] > 1:  
            data = tf.reduce_mean(data, axis=-1)  
        
        max_val = tf.reduce_max(tf.abs(data))
        if max_val > 0:
            data = data / max_val

        return data, sample_rate

    except Exception as e:
        print(f"Error loading WAV file '{file_path}': {e}")
        raise e

def process_directory_io(directory_path):
    wav_files = []

    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.lower().endswith(".wav"):
                wav_files.append(os.path.join(root, file))

    if not wav_files:
        print("No WAV files found in the directory.")
        return

    for file_path in wav_files:
        print(f"Processing file: {file_path}")
        try:
            audio, sample_rate = load_wav_file_io(file_path)
            print(f"Audio shape: {audio.shape}, Sample rate: {sample_rate}")
        except Exception as e:
            print(f"Failed to process file: {file_path}. Error: {e}")

In [5]:
def process_directory(directory_path):
    if 'tensorflow_io' in sys.modules:
        print("tensorflow_io ist importiert")
        process_directory_io(directory_path)
    else:
        print("tensorflow_io ist nicht importiert")
        process_directory_basic(directory_path)

In [ ]:
commands = np.array(tf.io.gfile.listdir(str(TRAIN_DIR)))
commands = commands[(commands != 'README.md') & (commands != '.DS_Store')]
print('Folder Created:', commands)

# Run Notebook individual

In [ ]:

TRAIN_DIR = pathlib.Path('data/train_files')
TEST_DIR = pathlib.Path('data/test_files')
DATA_DIR = pathlib.Path('data')
VAL_DIR = pathlib.Path('data/small_val_ds')



extract_zip(TRAIN_DIR, DATA_DIR)
print('-' * 50)  
extract_zip(TEST_DIR, DATA_DIR)
print('-' * 50)  
extract_zip(VAL_DIR, DATA_DIR)
print('-' * 50)
print('-' * 50)  
rename_audio_files(DATA_DIR)
print('-' * 50)
print('-' * 50)  
process_directory(TRAIN_DIR) 
process_directory(VAL_DIR) 